In [0]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# 第1章 線形回帰



## 1.2 重回帰


### p=1
特徴量が$p$ 個 $(p > 1)$である場合に，1.1の内容を拡張する．
まず，$p=1$の場合を行列で表示して復習する．
$y \in \mathbb{R}^N, X \in \mathrm{Mat}(2, N, \mathbb{R}), \beta \in \mathbb{R}^2$を
$$
    y = 
    \begin{pmatrix}
        y_1\\
        \cdots\\
        y_N
    \end{pmatrix},\,
    X = 
    \begin{pmatrix}
        1 & x_1\\
        \cdots & \cdots\\
        1 & x_N
    \end{pmatrix},\,
    \beta = 
    \begin{pmatrix}
        \beta_0\\
        \beta_1
    \end{pmatrix}
$$
とする．このとき，$L = \| y - X\beta\|^2$とかける．
よって$L$の勾配は，
$$
    \nabla L = \frac{\partial L }{\partial \beta} = 
    \begin{pmatrix}
        \frac{\partial L}{\partial \beta_0}\\
        \frac{\partial L}{\partial \beta_1}\\
    \end{pmatrix}
    = -2X^t(y - X\beta) 
$$

### p > 1
$y \in \mathbb{R}^N, X \in \mathrm{Mat}(p+1, N, \mathbb{R}), \beta \in \mathbb{R}^p$を
$$
    y = 
    \begin{pmatrix}
        y_1\\
        \cdots\\
        y_N
    \end{pmatrix},\,
    X = 
    \begin{pmatrix}
        1 & x_{11} & \cdots & x_{1p}\\
        \cdots & \cdots & \cdots\\
        1 & x_{N1} & \cdots & x_{Np}
    \end{pmatrix},\,
    \beta = 
    \begin{pmatrix}
        \beta_0\\
        \cdots\\
        \beta_p
    \end{pmatrix}
$$
とする．同様の議論で，$\nabla L = -2X^t(y - X\beta)$となる．$\nabla L = 0$とすると，
$X^ty = X^tX\beta$.$X^tX$が正則という仮定のもとで，両辺$(X^tX)^{-1}$をかけると，
$$
    \beta = (X^tX)^{-1}X^ty
$$
とかける．


### EX20

In [0]:
def generate_data(N, p, beta=np.array([])):
    """
        N: size of data
        p: the number of variables
        beta: variables

        Remark: 
        Substituting beta (i.e., the variable y generated by X@β + \epsilon) allows us to 
        see how exactly true the result generated by the above expression will be. 
        If not, you can check the formula against the random data.
    """
    X = np.insert(
        np.random.randn(N, p-1),
        0,
        1,
        axis=1
    )
    if beta.size == 0:
        beta = np.random.randn(p)
    y = X @ beta + np.random.randn(N)
    return y, X 

In [0]:
def find_beta(y, X):
    return np.linalg.inv(X.T @ X)@ X.T @ y

In [0]:
y, X = generate_data(10, 2)
find_beta(y, X)

array([0.28477933, 2.77915069])

In [0]:
y, X = generate_data(10, 3, np.array([1,2,3]))
find_beta(y, X)

array([1.51486549, 1.68739722, 2.88253961])

In [0]:
y, X = generate_data(10000, 5, np.array([1, 2, 3, 4, 5]))
find_beta(y, X)

array([1.00318275, 2.00763982, 3.0008875 , 4.00231577, 5.00978075])

### $X^tX$の正則性
$X^tX$ が逆行列を持たないという仮定について考えておく．

まず $X$ は正方行列であるとは限らないが，$X^tX$ は正方行列になることに注意する．上では，$X$ は $N \times (p+1)$ - 行列であるとしたから，$X^tX$ は $(p+1) \times (p+1)$ - 行列である．

$X^tX$ が逆行列を持つとすると，$\mathrm{rank} X^tX = p+1$であるが，
- $\mathrm{rank} X < \mathrm{min}\{N, p+1\}$
- $\mathrm{rank} X^tX < \mathrm{rank} X$

であるから，$N < p+1$ のときには $X^tX$ は逆行列を持たない．また，$X = (x_1, \dots, x_{p+1})$と表したとき，$x_i = x_j$となる$i, j (i \neq j)$が存在すると，$\mathrm{det}X = 0$である．このとき，$\mathrm{det}X^tX = \mathrm{det}X^t\mathrm{det}X = 0$ となるから $X^tX$は逆行列を持たない．

すなわち $N < p+1$または $X$ が同じ列（行）を持つとき，$X^tX$ は逆行列を持たない．

この状況が成り立つときはどんな時か．$X$ を特徴量のデータセットし，$y$をターゲットのデータとする．例えば，不動産の販売価格予想などを考える．このとき，$X$の各行は物件のデータに対応し，各列は築年数などの特徴量に対応していると思えば良い．$y$は販売価格である．

この時，$N < p+1$が成り立つとは，物件のデータ数よりも特徴が多いときである．また $X$ が同じ列を持つ時とは，同じ特徴量をデータセットが持つときである．したがってこういう場合には，この方法はうまくいかない．例えば擬似逆行列などの工夫をする必要がある．

